# OSM + ORS + DATA_MOS

## 1. Import libraries

In [ ]:
import osmnx as ox
import geopandas as gpd
import folium
import requests
import pandas as pd

## 2. Explore osmnx library

#### 2.1 Select territory 

In [ ]:
area = ox.geocode_to_gdf('Borovsk')

#### 2.2 Explore the boundaries we get

In [4]:
area.head()

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((36.44448 55.20016, 36.44454 55.19990...",55.240295,55.18143,36.560165,36.444484,308030994,relation,1919535,55.206719,36.481388,"Borovsk, Borovsky District, Kaluga Oblast, Cen...",boundary,administrative,0.599265


#### 2.3 List key-value pairs for tags and get geometries

In [4]:
tags = {'building': True}   
buildings = ox.geometries_from_place('Borovsk', tags)

buildings.head()

amenity cuisine              name note  \
element_type osmid                                              
way          100586666     NaN     NaN               NaN  NaN   
             100586671     NaN     NaN            Магнит  NaN   
             100586674     NaN     NaN     офисный центр  NaN   
             100586693     NaN     NaN               NaN  NaN   
             100586695     NaN     NaN  ТЦ "Коробейники"  NaN   

                                                                 geometry  \
element_type osmid                                                          
way          100586666  POLYGON ((36.48716 55.20558, 36.48730 55.20521...   
             100586671  POLYGON ((36.48712 55.20570, 36.48676 55.20565...   
             100586674  POLYGON ((36.48303 55.20665, 36.48299 55.20676...   
             100586693  POLYGON ((36.48299 55.20676, 36.48303 55.20665...   
             100586695  POLYGON ((36.48640 55.20771, 36.48613 55.20767...   

                           operator website tourism              shop  \
element_type osmid                                                      
way          100586666          NaN     NaN     NaN               NaN   
             100586671  АО "Тандер"     NaN     NaN       supermarket   
             100586674          NaN     NaN     NaN               NaN   
             100586693          NaN     NaN     NaN               NaN   
             100586695          NaN     NaN     NaN  department_store   

                       addr:housenumber  ... man_made leisure  bus  \
element_type osmid                       ...                         
way          100586666               11  ...      NaN     NaN  NaN   
             100586671                9  ...      NaN     NaN  NaN   
             100586674               16  ...      NaN     NaN  NaN   
             100586693               15  ...      NaN     NaN  NaN   
             100586695               27  ...      NaN     NaN  NaN   

                       public_transport fuel:lpg sport generator:method  \
element_type osmid                                                        
way          100586666              NaN      NaN   NaN              NaN   
             100586671              NaN      NaN   NaN              NaN   
             100586674              NaN      NaN   NaN              NaN   
             100586693              NaN      NaN   NaN              NaN   
             100586695              NaN      NaN   NaN              NaN   

                       generator:source fixme brand:en  
element_type osmid                                      
way          100586666              NaN   NaN      NaN  
             100586671              NaN   NaN      NaN  
             100586674              NaN   NaN      NaN  
             100586693              NaN   NaN      NaN  
             100586695              NaN   NaN      NaN  

[5 rows x 59 columns]

#### 2.4 Select only Polygon geometries

In [5]:
buildings = buildings.loc[buildings.geometry.type=='Polygon']

#### 2.5 Create web-map to look at the results

In [34]:
osm_map = folium.Map(location=[8.681495,49.41461], zoom_start=10)
points  = folium.features.GeoJson(buildings.to_json())
osm_map.add_child(points)
osm_map

## 3. ORS_API: directions, isochrones and more 

#### 3.1 Set API

In [25]:
ors_api = '5b3ce3597851110001cf6248d9a1e1f412b0477091ecc738bb12766c'

#### 3.2 Set parameters of locations, transportation mode and isochrones range (in seconds)

In [ ]:
transport_mode = 'driving-car'
data= {"locations":[[8.681495,49.41461],[8.686507,49.41943]],"range":[300,200]}
url = f'https://api.openrouteservice.org/v2/isochrones/{transport_mode}'
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': ors_api,
    'Content-Type': 'application/json; charset=utf-8'
}



: 

#### 3.3 Request isochrones

In [27]:
response = requests.post(url, json=data, headers=headers )

#### 3.4 Explore the result

In [30]:
response.json()

{'type': 'FeatureCollection',
 'bbox': [8.663618, 49.402738, 8.702732, 49.43552],
 'features': [{'type': 'Feature',
   'properties': {'group_index': 0,
    'value': 200.0,
    'center': [8.6814962950468, 49.41460087012079]},
   'geometry': {'coordinates': [[[8.672909, 49.417168],
      [8.672942, 49.417027],
      [8.674787, 49.408767],
      [8.67659, 49.407787],
      [8.676705, 49.407785],
      [8.676753, 49.407786],
      [8.686325, 49.412709],
      [8.690759, 49.413064],
      [8.690946, 49.413068],
      [8.691133, 49.413073],
      [8.691763, 49.413165],
      [8.693135, 49.417046],
      [8.692997, 49.417504],
      [8.692914, 49.417563],
      [8.690386, 49.418705],
      [8.684951, 49.420688],
      [8.680829, 49.422103],
      [8.677473, 49.422485],
      [8.677271, 49.42236],
      [8.67707, 49.422234],
      [8.673169, 49.419824],
      [8.672914, 49.417465],
      [8.672909, 49.417168]]],
    'type': 'Polygon'}},
  {'type': 'Feature',
   'properties': {'group_index': 0,

#### 3.5 Create DeoDataFrame 

In [31]:
gdf = gpd.GeoDataFrame.from_features(response.json()["features"])
print(gdf.head())

                                            geometry  group_index  value  \
0  POLYGON ((8.67291 49.41717, 8.67294 49.41703, ...            0  200.0   
1  POLYGON ((8.66362 49.41910, 8.66368 49.41875, ...            0  300.0   
2  POLYGON ((8.67626 49.41702, 8.67628 49.41683, ...            1  200.0   
3  POLYGON ((8.67080 49.41980, 8.67082 49.41944, ...            1  300.0   

                                  center  
0   [8.6814962950468, 49.41460087012079]  
1   [8.6814962950468, 49.41460087012079]  
2  [8.68650646264858, 49.41942995674008]  
3  [8.68650646264858, 49.41942995674008]  


#### 3.6 Create web-map to look at the results

In [33]:
osm_map = folium.Map(location=[8.681495,49.41461], zoom_start=10)
points  = folium.features.GeoJson(gdf.to_json())
osm_map.add_child(points)
osm_map

## 4. DataMos API: it was never so easy to get a data ...

#### 4.1 Set parameters for the request

In [41]:
datamos_api = 'b0c0ad1adf1319a3a8b1c2ed7649b21e'
data_set = 2263

url_data = f'https://apidata.mos.ru/v1/datasets/{data_set}/features?api_key={datamos_api}'

#### 4.2 Request data

In [42]:
data_mos = requests.get(url = url_data )

#### 4.2 Create GeoDataFrame

In [47]:
schools = gpd.GeoDataFrame.from_features(data_mos.json()["features"])
schools.head()


#### 4.3 Split attributes to a separate fields

In [49]:

schools_attributes = pd.DataFrame(schools['Attributes'].values.tolist(), index=schools.index)
schools_final = pd.concat([schools, schools_attributes], axis = 1).drop('Attributes', axis = 1)
schools_final.head()
